In [1]:
import sys
import xlsxwriter
import pandas as pd
import sys
import mysql.connector
import os
from dotenv import load_dotenv
from datetime import date

today = date.today()
load_dotenv()

#configurar la conexion a la base de datos
DB_USERNAME = os.getenv('DB_USERNAME')
DB_DATABASE = os.getenv('DB_DATABASE')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_PORT = os.getenv('DB_PORT')
DB_HOST=os.getenv('DB_HOST')

# Conectar a DB
cnx = mysql.connector.connect(user=DB_USERNAME,
                              password=DB_PASSWORD,
                              host=DB_HOST,
                              port=DB_PORT,
                              database=DB_DATABASE,
                              use_pure=False)
cursor=cnx.cursor(buffered=True)

In [2]:
egresados=pd.read_sql('select * from egresados where muestra=3',cnx)
telefonos=pd.read_sql('select * from telefonos',cnx)
recados=pd.read_sql('select * from recados',cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [3]:
ilocalizables=egresados.loc[egresados['status'].isin([6,11])]

In [4]:
len(ilocalizables)/len(egresados)

0.2726515742909186

In [5]:
becas_answers=pd.read_sql('select * from respuestas20 where nar3a is not null and nar3a!=1',cnx )

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [33]:
discrim=pd.read_sql('select * from discriminacion',cnx)

c:\Users\fgonzalez\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [6]:
becas_answers['nar3a'].unique()

array([ 2,  4,  3,  8,  6, 13, 10,  9, 15,  5, 11, 18, 19, 17, 16,  7, 14,
       12, 22, 20, 21], dtype=int64)

In [29]:
discrim['description'].unique()

array([None], dtype=object)

In [34]:
for i in range(len(discrim)):
    print(i)
    query=('insert into multiple_option_answers(encuesta_id,reactivo,clave_opcion) values(%s,%s,%s)')
    cursor.execute(query,tuple([str(discrim['encuesta_id'].values[i]),
               'nfr23',
                str(discrim['tipo'].values[i])]))
    # cnx.execute('insert')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [35]:
cnx.commit()
# cnx.close()

In [4]:
egresados.columns

Index(['id', 'cuenta', 'nombre', 'paterno', 'materno', 'carrera', 'plantel',
       'extras', 'promedio', 'correo', 'telefono', 'telefono2', 'anio_egreso',
       'correo2', 'bach', 'correo_unam', 'sistema', 'muestra', 'status',
       'llamadas', 'updated_at', 'sexo', 'fec_nac', 'actualized', 'fuente',
       'created_at'],
      dtype='object')

In [5]:
#Verificar ilocañlizables equivocado y no existe

ilocalizables=egresados.loc[(egresados['status'].isin([6]))&(egresados['muestra']==3)]

In [6]:
for i in range(len(ilocalizables)):
    eg_r=recados.loc[recados['cuenta']==str(ilocalizables['cuenta'].values[i])]
    if(len(eg_r)>1):
     
        if(11 in eg_r['status'].unique()):
           
           print(eg_r[['status','fecha']])
           print(' i ll take :', eg_r['status'].values[len(eg_r)-1])
           cursor.execute('update egresados set status= '+str(eg_r['status'].values[len(eg_r)-1])+' where id = '+str(ilocalizables['id'].values[i]))

       status               fecha
20470    11.0 2024-06-14 15:33:46
20472     6.0 2024-06-14 15:38:15
 i ll take : 6.0
       status               fecha
20914    11.0 2024-06-18 14:19:25
20915    11.0 2024-06-18 14:19:51
 i ll take : 11.0
       status               fecha
21023     7.0 2024-06-18 18:28:52
21121    11.0 2024-06-19 14:07:18
 i ll take : 11.0
       status               fecha
6488      7.0 2024-03-19 16:46:36
21135    11.0 2024-06-19 14:27:05
 i ll take : 11.0
       status               fecha
6215      7.0 2024-03-19 09:35:15
20535    11.0 2024-06-14 18:19:01
 i ll take : 11.0
       status               fecha
6447      7.0 2024-03-19 15:49:42
20976    11.0 2024-06-18 17:18:24
 i ll take : 11.0
       status               fecha
6395      7.0 2024-03-19 14:34:21
20963    11.0 2024-06-18 16:58:29
 i ll take : 11.0
       status               fecha
6441      7.0 2024-03-19 15:43:47
20973    11.0 2024-06-18 17:16:43
 i ll take : 11.0
       status               fecha
6433   

In [10]:
for i in range(len(ilocalizables)):
    eg_r=recados.loc[recados['cuenta']==str(ilocalizables['cuenta'].values[i])]
    if(len(eg_r)>=1):
     
        if(11 in eg_r['status'].unique()):
           
           print(eg_r[['status','fecha']])
           print(' i ll take :', eg_r['status'].values[len(eg_r)-1])
           cursor.execute('update egresados set status= '+str(eg_r['status'].values[len(eg_r)-1])+' where id = '+str(ilocalizables['id'].values[i]))

       status               fecha
18167    11.0 2024-06-03 17:26:42
 i ll take : 11.0
       status               fecha
18161    11.0 2024-06-03 17:12:30
 i ll take : 11.0
       status               fecha
19477    11.0 2024-06-10 17:50:16
 i ll take : 11.0
       status               fecha
19131    11.0 2024-06-07 15:51:15
 i ll take : 11.0
       status               fecha
18678    11.0 2024-06-05 17:16:15
 i ll take : 11.0
       status               fecha
18674    11.0 2024-06-05 17:12:06
 i ll take : 11.0
       status               fecha
18646    11.0 2024-06-05 15:59:28
 i ll take : 11.0
       status               fecha
18936    11.0 2024-06-06 17:06:10
 i ll take : 11.0
       status               fecha
19183    11.0 2024-06-07 17:26:09
 i ll take : 11.0
       status               fecha
18856    11.0 2024-06-06 15:19:09
 i ll take : 11.0
       status               fecha
18452    11.0 2024-06-04 17:26:33
 i ll take : 11.0
       status               fecha
19133    11.0 2024-0

In [23]:
recados.loc[recados['cuenta']==egresados['cuenta'].values[i]]

,id,cuenta,recado,fecha,status,updated_at,created_at,user_id,tel_id
0,38,416012248,Suena una vez y se corta,2024-01-18 12:19:49,7.0,2024-01-18 18:19:49,2024-01-18 18:19:49,10,40077
3,41,416012248,None,2024-01-18 12:23:21,7.0,2024-01-18 18:23:21,2024-01-18 18:23:21,10,40077


In [8]:
str(ilocalizables['id'].values[i])

'174962'

In [11]:
cnx.commit()

In [17]:
recados.columns

Index(['id', 'cuenta', 'recado', 'fecha', 'status', 'updated_at', 'created_at',
       'user_id', 'tel_id'],
      dtype='object')

In [15]:
len(ilocalizables)

832